window functions operate on a group of rows referred to as a window and calculate a return value for each row based on the group of rows 

window functions are useful for processing tasks such as 
- calculating a moving average
- computing a cumulative statistic
- accessing the value of rows given the relative position of the current row.



ranking functions:
- RANK
- DENSE_RANK
- PERCENT_RANK
- NTILE
- ROW_NUMBER

analytic functions:
- CUME_DIST
- LAG
- LEAD
- NTH_VALUE
- FIRST_VALUE
- LAST_VALUE

aggregate functions:
- MAX
- MIN
- COUNT
- SUM
- AVG

nulls_option:

specifies whether or not to skip null values when evaluating the window function

RESPECT NULLS means not skipping null values 

while IGNORE NULLS means skipping

if not specified the default is RESPECT NULLS

only LAG | LEAD | NTH_VALUE | FIRST_VALUE | LAST_VALUE can be used with IGNORE NULLS

In [1]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
from pyspark.sql import SparkSession, functions, Row
from pyspark.sql import functions
from pyspark.sql.functions import col, lit, concat, lower, upper, substring, min, max

In [2]:
import findspark
findspark.init()

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/25 14:03:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = [
    ("Lisa", "Sales", 10000, 35),
    ("Evan", "Sales", 32000, 38),
    ("Fred", "Engineering", 21000, 28),
    ("Alex", "Sales", 30000, 33),
    ("Tom", "Engineering", 23000, 33),
    ("Jane", "Marketing", 29000, 28),
    ("Jeff", "Marketing", 35000, 38),
    ("Paul", "Engineering", 29000, 23),
    ("Chloe", "Engineering", 23000, 25)
]

In [4]:
columns = ["name", "dept", "salary", "age"]

In [5]:
df = spark.createDataFrame(data, columns)

In [6]:
df.createOrReplaceTempView("employees")

In [7]:
spark.sql("SELECT * FROM employees")

name,dept,salary,age
Lisa,Sales,10000,35
Evan,Sales,32000,38
Fred,Engineering,21000,28
Alex,Sales,30000,33
Tom,Engineering,23000,33
Jane,Marketing,29000,28
Jeff,Marketing,35000,38
Paul,Engineering,29000,23
Chloe,Engineering,23000,25


In [8]:
spark.sql("SELECT name, dept, salary, RANK() OVER (PARTITION BY dept ORDER BY salary) AS rank FROM employees;")

name,dept,salary,rank
Fred,Engineering,21000,1
Tom,Engineering,23000,2
Chloe,Engineering,23000,2
Paul,Engineering,29000,4
Jane,Marketing,29000,1
Jeff,Marketing,35000,2
Lisa,Sales,10000,1
Alex,Sales,30000,2
Evan,Sales,32000,3


In [ ]:
spark.sql("SELECT name, dept, salary, DENSE_RANK() OVER (PARTITION BY dept ORDER BY salary) AS dense_rank FROM employees;")

name,dept,salary,rank
Fred,Engineering,21000,1
Tom,Engineering,23000,2
Chloe,Engineering,23000,2
Paul,Engineering,29000,3
Jane,Marketing,29000,1
Jeff,Marketing,35000,2
Lisa,Sales,10000,1
Alex,Sales,30000,2
Evan,Sales,32000,3


In [11]:
spark.sql("SELECT name, dept, salary, CUME_DIST() OVER (PARTITION BY dept ORDER BY salary) AS cume_dist FROM employees;")

name,dept,salary,cume_dist
Fred,Engineering,21000,0.25
Tom,Engineering,23000,0.75
Chloe,Engineering,23000,0.75
Paul,Engineering,29000,1.0
Jane,Marketing,29000,0.5
Jeff,Marketing,35000,1.0
Lisa,Sales,10000,0.3333333333333333
Alex,Sales,30000,0.6666666666666666
Evan,Sales,32000,1.0


In [13]:
spark.sql("SELECT name, dept, salary, ROW_NUMBER() OVER (PARTITION BY dept ORDER BY salary) AS row_number FROM employees;")

name,dept,salary,row_number
Fred,Engineering,21000,1
Tom,Engineering,23000,2
Chloe,Engineering,23000,3
Paul,Engineering,29000,4
Jane,Marketing,29000,1
Jeff,Marketing,35000,2
Lisa,Sales,10000,1
Alex,Sales,30000,2
Evan,Sales,32000,3


In [ ]:
# LAG(column,offset,default_value)
# LEAD(column,offset,default_value)

spark.sql("SELECT name, salary, LAG(salary, 1, 0) OVER (PARTITION BY dept ORDER BY salary) AS previous_salary, LEAD(salary, 1, 0) OVER (PARTITION BY dept ORDER BY salary) AS next_salary FROM employees;")

name,salary,previous_salary,next_salary
Fred,21000,0,23000
Tom,23000,21000,23000
Chloe,23000,23000,29000
Paul,29000,23000,0
Jane,29000,0,35000
Jeff,35000,29000,0
Lisa,10000,0,30000
Alex,30000,10000,32000
Evan,32000,30000,0


In [15]:
spark.sql("SELECT name, dept, salary, FIRST_VALUE(salary) OVER (PARTITION BY dept ORDER BY salary) AS first_salary FROM employees;")

name,dept,salary,first_salary
Fred,Engineering,21000,21000
Tom,Engineering,23000,21000
Chloe,Engineering,23000,21000
Paul,Engineering,29000,21000
Jane,Marketing,29000,29000
Jeff,Marketing,35000,29000
Lisa,Sales,10000,10000
Alex,Sales,30000,10000
Evan,Sales,32000,10000


In [17]:
spark.sql("SELECT name, dept, salary, LAST_VALUE(salary) OVER (PARTITION BY dept ORDER BY salary ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_salary FROM employees;")

name,dept,salary,last_salary
Fred,Engineering,21000,29000
Tom,Engineering,23000,29000
Chloe,Engineering,23000,29000
Paul,Engineering,29000,29000
Jane,Marketing,29000,35000
Jeff,Marketing,35000,35000
Lisa,Sales,10000,32000
Alex,Sales,30000,32000
Evan,Sales,32000,32000
